In [36]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import re
import warnings
warnings.filterwarnings('ignore')

In [37]:
np.random.seed(111)

In [38]:
# 加载数据集
print("正在加载数据...")
train_data = pd.read_csv('ruc_Class25Q1_train.csv')
test_data = pd.read_csv('ruc_Class25Q1_test.csv')
submission_template = pd.read_csv('submission_template_Class25Q1.csv', usecols=['ID'])
print(f"训练集样本数: {train_data.shape[0]}, 特征数: {train_data.shape[1]}")
print(f"测试集样本数: {test_data.shape[0]}, 特征数: {test_data.shape[1]}")
print("\n训练集缺失值统计:")
print(train_data.isnull().sum())

正在加载数据...
训练集样本数: 84133, 特征数: 31
测试集样本数: 14786, 特征数: 31

训练集缺失值统计:
城市          0
区域          0
板块          0
环线      41407
小区名称        0
价格          0
房屋户型      605
所在楼层        0
建筑面积        0
套内面积    58987
房屋朝向        0
建筑结构      605
装修情况      605
梯户比例     1695
配备电梯     8315
别墅类型    83384
交易时间        0
交易权属        0
上次交易    28953
房屋用途        2
房屋年限    29782
产权所属        0
抵押信息    84133
房屋优势    16064
核心卖点    16366
户型介绍    63671
周边配套    34027
交通出行    32437
lon         0
lat         0
年份          0
dtype: int64


In [39]:
# 定义正则表达式用于提取数值
AREA_PATTERN = re.compile(r'([\d\.]+)')
FLOOR_PATTERN = re.compile(r'共(\d+)层')
ROOM_PATTERNS = {
    '室': re.compile(r'(\d+)室'),
    '厅': re.compile(r'(\d+)厅'),
    '厨': re.compile(r'(\d+)厨'),
    '卫': re.compile(r'(\d+)卫')
}

In [40]:
# 特征工程函数：对数据进行清洗、衍生特征构造
def preprocess_data(df, is_train=True):
    df_copy = df.copy(deep=False)
    if is_train:
        y = df_copy['价格'].values  # 使用.values转为numpy数组，提高性能
        df_copy.drop('价格', axis=1, inplace=True)#从训练集中提取房价，并从特征中删除这个标签列，防止信息泄露
    else:
        y = None 
    has_house_type = '房屋户型' in df_copy.columns
    has_trade_time = '交易时间' in df_copy.columns
    has_house_age = '房屋年限' in df_copy.columns
    has_geo = 'lon' in df_copy.columns and 'lat' in df_copy.columns #检查数据集中是否同时包含 'lon'（经度）和 'lat'（纬度）两个字段
    has_id = 'ID' in df_copy.columns
    df_copy['建筑面积_数值'] = df_copy['建筑面积'].str.extract(AREA_PATTERN).astype(float)
    df_copy['套内面积_数值'] = df_copy['套内面积'].str.extract(AREA_PATTERN).astype(float)
    mask = (df_copy['建筑面积_数值'].notna()) & (df_copy['套内面积_数值'].notna()) #创建一个布尔掩码（mask），判断哪些行的这两个面积值都不是缺失，从而可以计算公摊比例
    df_copy.loc[mask, '公摊比例'] = (df_copy.loc[mask, '建筑面积_数值'] - df_copy.loc[mask, '套内面积_数值']) / df_copy.loc[mask, '建筑面积_数值']
    directions = ['南', '北', '东', '西']
    for direction in directions:
        df_copy[f'朝{direction}'] = df_copy['房屋朝向'].str.contains(direction).fillna(False).astype(np.int8)  #  创建“朝某方向”变量：若“房屋朝向”字段中包含该方向，
                                                                                                               #则为1，否则为0
    
    df_copy['南北通透'] = ((df_copy['朝南'] == 1) & (df_copy['朝北'] == 1)).astype(np.int8) # 构造“南北通透”变量：如果房屋同时朝南又朝北，则为1，否则为0
    floor_types = [('高', '高楼层'), ('中', '中楼层'), ('低', '低楼层'), ('底', '底层'), ('顶', '顶层')]
    for prefix, pattern in floor_types:  # 遍历每种楼层类型，创建哑变量特征
        df_copy[f'楼层_{prefix}'] = df_copy['所在楼层'].str.contains(pattern).fillna(False).astype(np.int8)  # 判断“所在楼层”字段是否包含对应楼层关键词，是则为1，否则为0
    df_copy['总楼层数'] = df_copy['所在楼层'].str.extract(FLOOR_PATTERN).astype(float) # 提取“所在楼层”字段中“共N层”部分的数字（总楼层数），并转换为 float 类型
    for decoration in ['精装', '简装', '毛坯']: # 对“装修情况”字段进行哑变量编码（精装 / 简装 / 毛坯）
        df_copy[f'{decoration}修' if decoration != '毛坯' else '毛坯房'] = (df_copy['装修情况'] == decoration).astype(np.int8) # 对每个装修类型生成0/1哑变量：是该类型为1，否则为0
    df_copy['有电梯'] = (df_copy['配备电梯'] == '有').astype(np.int8)  
    for structure in ['钢混结构', '砖混结构', '混合结构']:
        df_copy[structure] = (df_copy['建筑结构'] == structure).astype(np.int8)
    if has_trade_time:   # 如果数据中包含“交易时间”字段，则提取时间特征
        trade_dates = pd.to_datetime(df_copy['交易时间'], cache=True)
        df_copy['交易年份'] = trade_dates.dt.year
        df_copy['交易月份'] = trade_dates.dt.month
        df_copy['交易季度'] = trade_dates.dt.quarter
    

    if has_house_age: # 如果数据中包含“房屋年限”字段
        df_copy['满五年'] = (df_copy['房屋年限'] == '满五年').astype(np.int8)
        df_copy['满两年'] = (df_copy['房屋年限'] == '满两年').astype(np.int8)
    # 定义环线等级映射关系：越靠近市中心，数值越小，越靠外环数值越大
    环线映射 = {         
        '一环以内': 0,
        '一至二环': 1,
        '二至三环': 2,
        '三至四环': 3,
        '四至五环': 4,
        '五至六环': 5,
        '六环以外': 6
    }
    df_copy['环线_数值'] = df_copy['环线'].map(环线映射).fillna(-1) # 将原始文本型“环线”字段映射为数值型变量，缺失值填为 -1

    if has_house_type:  # 如果数据中包含“房屋户型”字段（如 '3室1厅1厨1卫'）
        n_rows = len(df_copy) # 获取数据行数，用于初始化变量数组
        室数 = np.zeros(n_rows) # 初始化结构特征数组
        厅数 = np.zeros(n_rows)
        厨数 = np.zeros(n_rows)
        卫数 = np.zeros(n_rows)
        mask = df_copy['房屋户型'].notna() # 创建布尔掩码：筛选出“房屋户型”字段不为空的行
        valid_types = df_copy.loc[mask, '房屋户型'].values  # 提取有效的“房屋户型”字符串
        valid_indices = np.where(mask)[0] # 获取对应的索引位置，便于后续赋值
        
        for i, 户型 in zip(valid_indices, valid_types):
            for room_type, pattern in ROOM_PATTERNS.items(): # 针对每种房间类型（室/厅/厨/卫），使用正则提取数字
                match = pattern.search(户型)
                if match:
                    if room_type == '室':
                        室数[i] = int(match.group(1))
                    elif room_type == '厅':
                        厅数[i] = int(match.group(1))
                    elif room_type == '厨':
                        厨数[i] = int(match.group(1))
                    elif room_type == '卫':
                        卫数[i] = int(match.group(1))
        
        df_copy['室数'] = 室数
        df_copy['厅数'] = 厅数
        df_copy['厨数'] = 厨数
        df_copy['卫数'] = 卫数
        df_copy['房间总数'] = 室数 + 厅数 + 厨数 + 卫数
        
        # # 创建“居室类型”哑变量，用于表示房屋属于哪一类居室
        df_copy['一居室'] = (df_copy['室数'] == 1).astype(np.int8)
        df_copy['二居室'] = (df_copy['室数'] == 2).astype(np.int8)
        df_copy['三居室'] = (df_copy['室数'] == 3).astype(np.int8)
        df_copy['四居室及以上'] = (df_copy['室数'] >= 4).astype(np.int8)
    
    # 添加交互特征和非线性特征
    df_copy['面积_环线'] = df_copy['建筑面积_数值'] * df_copy['环线_数值'] # 面积 × 环线：反映面积在不同地段的价值变化（中心 vs 外环）
    df_copy['面积_南北通透'] = df_copy['建筑面积_数值'] * df_copy['南北通透'] # 面积 × 南北通透：反映好户型（南北通透）在不同面积段的溢价效果
    df_copy['建筑面积_平方'] = df_copy['建筑面积_数值'] ** 2 # 面积的平方项：考虑非线性影响（如大面积溢价递减或递增）
    df_copy['环线_平方'] = df_copy['环线_数值'] ** 2 # 环线数值的平方项：反映地段对价格影响是否是线性的
    

    if has_geo: 

        市中心_lon, 市中心_lat = 116.397128, 39.916527  # 如果包含经纬度信息（lon/lat），则计算房屋到北京市中心的距离
        df_copy['到市中心距离'] = np.sqrt(
            (df_copy['lon'] - 市中心_lon)**2 + 
            (df_copy['lat'] - 市中心_lat)**2
        )
    # 明确列出模型训练所需的关键特征字段，其他字段将被丢弃
    features_to_keep = [
        '城市', '区域', '板块', '环线_数值', '小区名称',  # 区域地理信息
        '建筑面积_数值', '套内面积_数值', '公摊比例',     # 面积类特征
        '朝南', '朝北', '朝东', '朝西', '南北通透',      # 朝向类特征
        '楼层_高', '楼层_中', '楼层_低', '楼层_底', '楼层_顶', '总楼层数', # 楼层相关
        '精装修', '简装修', '毛坯房', '有电梯',   # 装修、电梯
        '钢混结构', '砖混结构', '混合结构',        # 建筑结构
        '面积_环线', '面积_南北通透', '建筑面积_平方', '环线_平方'# 派生交互 & 非线性特征
    ]  
    # 如果数据中有交易时间字段，则保留交易时间拆分特征
    if has_trade_time:
        features_to_keep.extend(['交易年份', '交易月份', '交易季度'])
    # 如果有房屋年限字段，则保留满两年/满五年特征
    if has_house_age:
        features_to_keep.extend(['满五年', '满两年'])
    # 如果包含房屋户型信息，则保留结构化房型特征及居室分类
    if has_house_type:
        features_to_keep.extend(['室数', '厅数', '厨数', '卫数', '房间总数', 
                                '一居室', '二居室', '三居室', '四居室及以上'])
    # 如果包含经纬度字段，则保留“到市中心距离”这个地理特征
    if has_geo:
        features_to_keep.extend(['到市中心距离'])
    # 如果有 ID 字段（用于测试集结果对齐），也保留
    if has_id:
        features_to_keep.append('ID')
    # 仅保留最终筛选的特征列，构建建模数据集
    df_processed = df_copy[features_to_keep]
    # 返回预处理后的特征数据（df_processed）和标签（y）
    return df_processed, y

In [41]:
#预处理数据 
print("\n正在预处理数据...")
X_processed, y = preprocess_data(train_data, is_train=True) # 对训练集进行预处理，提取特征 X 和目标变量 y
X_test_processed, _ = preprocess_data(test_data, is_train=False) # 对测试集进行预处理（测试集没有 y 值，返回 None）


正在预处理数据...


In [42]:
#删除高度相关特征 
def remove_correlated_features(X, threshold=0.85): 
    numeric_X = X.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64'])  # 仅选取数值型变量用于计算相关性
    corr_matrix = numeric_X.corr().abs()   # 计算皮尔逊相关系数矩阵（取绝对值，忽略正负方向）
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  # 提取矩阵的上三角部分，避免重复计算（对称矩阵）
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)] # 找出任意一列中与其他特征的相关系数大于阈值的列，准备移除
    print(f"移除以下高度相关的特征: {to_drop}") 
    return X.drop(to_drop, axis=1)

In [43]:
# 划分训练/验证集 
X_filtered = remove_correlated_features(X_processed)
if 'ID' in X_processed.columns and 'ID' not in X_filtered.columns:
    X_filtered['ID'] = X_processed['ID']
X_test_filtered = X_test_processed[X_filtered.columns]
print("\n正在分割训练集和验证集...")
X_train, X_val, y_train, y_val = train_test_split(
    X_filtered, y, test_size=0.2, random_state=111
)

移除以下高度相关的特征: ['套内面积_数值', '南北通透', '面积_南北通透', '环线_平方', '交易季度', '房间总数']

正在分割训练集和验证集...


In [44]:
# 数值与分类特征预处理管道
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' and col != 'ID'] # 识别出类别特征：类型是 object，并排除 'ID' 字段
numeric_cols = [col for col in X_train.columns if col not in categorical_cols and col != 'ID'] # 数值特征：不是类别变量，也不是 ID
numeric_transformer = Pipeline(steps=[         # 数值特征的处理流程：缺失值填充（中位数）+ 标准化（均值为0，方差为1）
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[             # 类别特征的处理流程：缺失值填充（众数）+ OneHot编码
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(                  # 整合数值与类别特征的整体预处理器（列变换器）
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    verbose=0
)

In [45]:
# 建立模型管道
ridge_model = Ridge(alpha=1.0, solver='auto')
# lasso_model = Lasso(alpha=1.0, max_iter=1000)
#linear_model = LinearRegression()
#elastic_net_model = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=1000, random_state=111)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ridge_model)
    #('regressor', linear_model)
    # ('regressor', lasso_model)
    #('regressor', elastic_net_model)
])
print("\n正在训练岭回归模型...")
#print("\n正在训练线性回归模型...")
# print("\n正在训练Lasso回归模型...")
#print("\n正在训练弹性网络回归模型...")

model.fit(X_train, y_train)


正在训练岭回归模型...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['城市', '区域', '板块', '环线_数值',
                                                   '建筑面积_数值', '公摊比例', '朝南',
                                                   '朝北', '朝东', '朝西', '楼层_高',
                                                   '楼层_中', '楼层_低', '楼层_底',
                                                   '楼层_顶', '总楼层数', '精装修', '简装修',
                                                   '毛坯房', '有电梯', '钢混结构', '砖混结构',
                                                   '混合结构', '面积_环线', '建筑面积_平方',
                                                   '交易年份', '交易月份', '满五年', '满两年',
                                                   '室数', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['小区名称'])],
                                   verbose=0)),
                ('regressor', Ridge())])

In [46]:
# 模型评估函数
def evaluate_model(model, X, y, dataset_name):
    """评估模型性能并打印结果"""
    y_pred = model.predict(X)

    errors = np.abs(y - y_pred)
    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(errors**2))
    median_error = np.median(errors)
    
    print(f"{dataset_name} 性能评估:")
    print(f"平均绝对误差 (MAE): {mae:.2f}")
    print(f"均方根误差 (RMSE): {rmse:.2f}")
    print(f"绝对中位数误差: {median_error:.2f}")
    
    return mae, rmse, median_error

In [47]:
print("\n正在评估模型性能...")
train_mae, train_rmse, train_median = evaluate_model(model, X_train, y_train, "训练集")
val_mae, val_rmse, val_median = evaluate_model(model, X_val, y_val, "验证集")

# 交叉验证 
print("\n正在执行6折交叉验证...")
cv = KFold(n_splits=6, shuffle=True, random_state=111)

cv_scores = cross_val_score(model, X_filtered, y, cv=cv, scoring='neg_mean_absolute_error')
cv_mae = -cv_scores
cv_rmse = np.sqrt(-cross_val_score(model, X_filtered, y, cv=cv, scoring='neg_mean_squared_error'))

print("交叉验证结果:")
print(f"平均 MAE: {cv_mae.mean():.2f} (标准差: {cv_mae.std():.2f})")
print(f"平均 RMSE: {cv_rmse.mean():.2f} (标准差: {cv_rmse.std():.2f})")

# 将RMSE结果保存到Excel文件
import pandas as pd
rmse_results = pd.DataFrame({
    '数据集': ['训练集', '验证集', '交叉验证'],
    'RMSE': [train_rmse, val_rmse, cv_rmse.mean()]
})
rmse_results.to_excel('岭回归_rmse_results.xlsx', index=False)
#rmse_results.to_excel('线性回归_rmse_results.xlsx', index=False)
# rmse_results.to_excel('Lasso回归_rmse_results.xlsx', index=False)
#rmse_results.to_excel('弹性网络回归_rmse_results.xlsx', index=False)
print("\n已将RMSE结果保存到xlsx文件中")
print("\n正在对测试集进行预测...")
model.fit(X_filtered, y)
test_predictions = model.predict(X_test_filtered)


正在评估模型性能...
训练集 性能评估:
平均绝对误差 (MAE): 370667.74
均方根误差 (RMSE): 829053.52
绝对中位数误差: 194763.09
验证集 性能评估:
平均绝对误差 (MAE): 392407.58
均方根误差 (RMSE): 931968.73
绝对中位数误差: 200092.43

正在执行6折交叉验证...
交叉验证结果:
平均 MAE: 397451.11 (标准差: 7632.31)
平均 RMSE: 1079864.43 (标准差: 267569.42)

已将RMSE结果保存到xlsx文件中

正在对测试集进行预测...


In [48]:
submission = pd.DataFrame()
submission['ID'] = X_test_processed['ID'] if 'ID' in X_test_processed.columns else test_data['ID']
submission['Price'] = test_predictions
print("\n正在保存预测结果...")
submission.to_csv('prediction.csv', index=False)
print(f"预测结果数量为: {len(submission)} 条")
# 用 IQR 方法移除预测结果中的异常值 
q1 = np.percentile(submission['Price'], 25)
q3 = np.percentile(submission['Price'], 75)
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

print(f"预测值的IQR范围：[{lower_bound:.2f}, {upper_bound:.2f}]")

# 筛选出非异常值
filtered_submission = submission[
    (submission['Price'] >= lower_bound) & (submission['Price'] <= upper_bound)
]

# 保存新的 prediction 文件
filtered_submission.to_csv('prediction_filtered.csv', index=False)

print(f"✅ 去除异常值后的预测结果数量为: {len(filtered_submission)} 条")

print("\n任务完成!")


if hasattr(model['regressor'], 'coef_'):

    feature_names = []

    for name, transformer, features in model['preprocessor'].transformers_:
        if name == 'num':
            feature_names.extend(features)
        elif name == 'cat':
          
            try:
                feature_names.extend(
                    model['preprocessor'].named_transformers_['cat']['onehot'].get_feature_names_out(features).tolist()
                )
            except:
        
                n_cats = len(model['preprocessor'].named_transformers_['cat']['onehot'].categories_)
                feature_names.extend([f'cat_{i}' for i in range(n_cats)])
    

    importance = np.abs(model['regressor'].coef_)
    

    if len(feature_names) == len(importance):

        sorted_idx = np.argsort(-importance)
        top_features = [feature_names[i] for i in sorted_idx[:20]]
        top_importance = importance[sorted_idx[:20]]
        
        print("\n特征重要性（前20个）:")
        for f, imp in zip(top_features, top_importance):
            print(f"{f}: {imp:.6f}")
    else:
        print(f"\n特征名称和系数长度不匹配: {len(feature_names)} vs {len(importance)}")


正在保存预测结果...
预测结果数量为: 14786 条
预测值的IQR范围：[-2337973.88, 6327466.09]
✅ 去除异常值后的预测结果数量为: 13797 条

任务完成!

特征重要性（前20个）:
小区名称_万科翡翠滨江(一期): 25344289.297712
小区名称_西郊大公馆: 24279783.270907
小区名称_京润水上花园别墅: 20695120.226743
小区名称_大华西郊别墅: 19941516.235852
小区名称_九庐: 18681860.589678
小区名称_泰禾中国院子: 18563077.655126
小区名称_中海紫御豪庭(公寓): 18461797.606521
小区名称_湖畔佳苑(别墅): 18270173.149142
小区名称_西郊紫郡: 16971341.991974
小区名称_静鼎安邦府邸(别墅): 16145654.665476
小区名称_瑞虹新城悦庭: 15720402.748941
小区名称_优山美地B区: 14151747.643726
小区名称_一瓶: 13873224.250423
小区名称_仁恒河滨城(二期): 13852224.147254
小区名称_泛海容郡: 13589638.393553
小区名称_仁恒河滨花园: 13201899.398682
小区名称_懋源·璟玺: 13127800.527905
小区名称_如园北区: 12676540.170281
小区名称_久事西郊名墅: 12554935.236087
小区名称_佘山银湖别墅: 12235406.618124
